In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2 as cv
import warnings
from collections import OrderedDict
import dlib
import imutils
import matplotlib

In [2]:
warnings.filterwarnings(action='ignore')

In [23]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat" "/content/shape_predictor_68_face_landmarks.dat"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def cheek(img) :
    CHEEK_IDXS = OrderedDict([("left_cheek", (1, 2, 3, 4, 5, 48, 31)),
                            ("right_cheek", (11, 12, 13, 14, 15, 35, 54))
                            ])

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    img = imutils.resize(img, width=600)

    overlay = img.copy()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    bounding_rects = []
    detections = detector(gray, 0)
    for k, d in enumerate(detections):
        shape = predictor(gray, d)
        for (_, name) in enumerate(CHEEK_IDXS.keys()):
            pts = np.zeros((len(CHEEK_IDXS[name]), 2), np.int32)
            for i, j in enumerate(CHEEK_IDXS[name]):
                pts[i] = [shape.part(j).x, shape.part(j).y]
            pts = pts.reshape((-1, 1, 2))
            cv.drawContours(overlay, [pts], -1, (255, 255, 255), -1, cv.LINE_AA)
            bounding_rects.append(cv.boundingRect(pts))
    height, width=overlay.shape[:2]

    for y in range(0, height):
        for x in range(0, width):
            b=overlay.item(y, x, 0)
            g=overlay.item(y, x, 1)
            r=overlay.item(y, x, 2)
            if b==255 and g==255 and r==255:
                continue
            overlay.itemset(y, x, 0, 0)
            overlay.itemset(y, x, 1, 0)
            overlay.itemset(y, x, 2, 0)
    output = 0
    output=cv.bitwise_and(img, overlay, output)
    black=0
    for y in range(0, height):
        for x in range(0, width):
            b=output.item(y, x, 0)
            g=output.item(y, x, 1)
            r=output.item(y, x, 2)
            if b==0 and g==0 and r==0:
                black+=1
            if b==255 and g==255 and r==255:
                output.itemset(y, x, 0, 0)
                output.itemset(y, x, 1, 0)
                output.itemset(y, x, 2, 0)
    if black > height*width-10 : return img
    return output


In [8]:
class Color :
    person_HSV = []

    def color_classifier(self, person_HSV) :
        self.H = float(person_HSV[0])
        self.S = float(person_HSV[1])
        self.V = float(person_HSV[2])
        diff = round(self.V - self.S, 2)
        print(self.H, diff)
        if self.H >= 20 and self.H <= 210 :
            if diff >= 68.25 :
                    self.ans = 0
                    # Spring
            elif diff < 68.25:
                    self.ans = 1
                    # Autumn
        elif (self.H >= 0 and self.H < 20) or (self.H > 210 and self.H < 360) :
            if diff >= 68.75 :
                    self.ans = 2
                    # Summer
            elif diff < 68.75:
                    self.ans = 3
                    # Winter

        else :
            self.ans = -1
            # 에러

        return self.ans


In [9]:
def color_ratio(clt) :
    numLabels = np.arange(0, len(np.unique(clt.labels_))+1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist

In [10]:
def plot_colors(hist, centroids):
    bar = np.zeros((50, 300, 3), dtype = "uint8")
    startX = 0

    for (percent, color) in zip(hist, centroids):
        endX = startX + (percent * 300)
        cv.rectangle(bar, (int(startX), 0), (int(endX), 50),
            color.astype("uint8").tolist(), -1)
        startX = endX
    return bar

In [16]:
def skin_detector(img, file_name) :
    lower = np.array([0, 48, 80], dtype = "uint8")
    upper = np.array([20, 255, 255], dtype = "uint8")

    converted = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    skinMask = cv.inRange(converted, lower, upper)

    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (11,11))
    skinMask = cv.erode(skinMask, kernel, iterations = 2)
    skinMask = cv.dilate(skinMask, kernel, iterations = 2)

    skinMask = cv.GaussianBlur(skinMask, (3,3), 0)
    skin = cv.bitwise_and(img, img, mask = skinMask)

    result = skin
    img = cv.cvtColor(result, cv.COLOR_BGR2HLS)
    skin_img = img
    temp_img = cv.cvtColor(img, cv.COLOR_HLS2RGB)

    h, w, c = img.shape

    for i in range(h) :
        for j in range(w) :
            H = img[i][j][0]
            L = img[i][j][1]
            S = img[i][j][2]

            R = temp_img[i][j][0]
            G = temp_img[i][j][1]
            B = temp_img[i][j][2]

            if R==0 and G==0 and B==0:
                continue

            LS_ratio = L/S
            skin_pixel = bool((S>=50) and (LS_ratio > 0.5) and (LS_ratio < 3.0) and ((H <= 25) or (H >= 165)))
            temp_pixel = bool((R == G) and (G == B) and (R >= 220))

            if skin_pixel :
                if temp_pixel :
                    skin_img[i][j][0] = 0
                    skin_img[i][j][1] = 0
                    skin_img[i][j][2] = 0
                else :
                    pass
            else :
                skin_img[i][j][0] = 0
                skin_img[i][j][1] = 0
                skin_img[i][j][2] = 0

    skin_img = cv.cvtColor(skin_img, cv.COLOR_HLS2BGR)
    for i in range(h) :
        for j in range(w) :
            B = skin_img[i][j][0]
            G = skin_img[i][j][1]
            R = skin_img[i][j][2]

            bg_pixel = bool(B==0 and G==0 and R==0)

            if bg_pixel :
                skin_img[i][j][0] = 255
                skin_img[i][j][1] = 255
                skin_img[i][j][2] = 255
            else :
                pass

    cvt_img = cv.cvtColor(skin_img, cv.COLOR_BGR2RGB)
    cvt_img = cvt_img.reshape((cvt_img.shape[0]*cvt_img.shape[1], 3))
    k = 20
    clt = KMeans(n_clusters=k)
    clt.fit(cvt_img)


    hist = color_ratio(clt)
    temp = np.array(clt.cluster_centers_)

    del_index = hist.argmax()
    hist = np.delete(hist, del_index)
    temp = np.delete(temp, del_index, 0)


    try :
        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)
    except ValueError :
        print(file_name, "에러")
        pass

    hist = hist / hist.sum()
    bar = plot_colors(hist, temp)

    bar = cv.cvtColor(bar, cv.COLOR_BGR2RGB)

    return bar

In [11]:
def color_convert2(cheek):
    img=cheek
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

    R = []
    G = []
    B = []
    for i in img :
        for j in i :
            R.append(j[0])
            G.append(j[1])
            B.append(j[2])

    R_sum = 0
    G_sum = 0
    B_sum = 0

    # 각 R, G, B의 합계 구하기
    for i in range(len(R)) :
        R_sum += R[i]
        G_sum += G[i]
        B_sum += B[i]

    R_avg = int(round((R_sum / len(R)), 0))  # R값 평균
    G_avg = int(round((G_sum / len(G)), 0))  # G값 평균
    B_avg = int(round((B_sum / len(B)), 0))  # B값 평균

    img_avg = img

    for i in img_avg :
        for j in i :
            j[0] = R_avg
            j[1] = G_avg
            j[2] = B_avg

    bgr_img_avg = cv.cvtColor(img_avg, cv.COLOR_RGB2BGR)
    bgr_img_avg=cv.resize(bgr_img_avg, (50, 50))
    return bgr_img_avg


In [14]:
def color_convert(cheek) :
    img=cheek
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

    sum = 0
    R = []
    G = []
    B = []
    for i in img :
        for j in i :
            R.append(j[0])
            G.append(j[1])
            B.append(j[2])

    R_sum = 0
    G_sum = 0
    B_sum = 0

    # 각 R, G, B의 합계 구하기
    for i in range(len(R)) :
        R_sum += R[i]
        G_sum += G[i]
        B_sum += B[i]

    R_avg = int(round((R_sum / len(R)), 0))  # R값 평균
    G_avg = int(round((G_sum / len(G)), 0))  # G값 평균
    B_avg = int(round((B_sum / len(B)), 0))  # B값 평균
    RGB_color = [R_avg, G_avg, B_avg]

    arr_RGB_color = np.array(RGB_color)
    float_arr_RGB_color = arr_RGB_color / 255
    float_tp_RGB_color = tuple(float_arr_RGB_color)
    HSV_color = matplotlib.colors.rgb_to_hsv(float_tp_RGB_color)
    HSV_color2 = np.array([round(HSV_color[0]*359, 3), round(HSV_color[1] * 100, 3)-4, round(HSV_color[2] * 100, 3)+8])
    HSV_color2 = list(HSV_color2)
    HSV_color2[0] = round(HSV_color2[0], 2)
    HSV_color2[1] = round(HSV_color2[1], 2)
    HSV_color2[2] = round(HSV_color2[2], 2)
    return HSV_color2


In [25]:
def save_img(file_name, skin_type) :

    if skin_type == 0 :
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

img = cv.imread("/content/drive/MyDrive/사계절_연예인 이미지 데이터셋/prototype/웜톤_jpg/공효진.jpg") # 퍼스널컬러 알고싶은 사진
file_name="personal"
cheekimg = cheek(img)
cv.imwrite("cheek.jpg", cheekimg)
bar = skin_detector(cheekimg, file_name)
cv.imwrite("bar.jpg", bar)
bgr = color_convert2(bar)
hsv = color_convert(bar)

color_class = Color()
skin_type = color_class.color_classifier(hsv)
cv.imwrite("skin.jpg", bgr)
save_img(file_name, skin_type)

20.94 62.59
autumn


1. 연예인 이미지(웜톤)

In [27]:
import os
import cv2 as cv

def save_img(file_name, skin_type) :

    if skin_type == 0 :
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

# 폴더 경로 지정
folder_path = "/content/drive/MyDrive/사계절_연예인 이미지 데이터셋/prototype/웜톤_jpg/"

# 폴더 내의 모든 파일을 가져오기 위해 glob 사용
import glob
file_list = glob.glob(folder_path + "*.jpg") + glob.glob(folder_path + "*.jpeg")

for file_path in file_list:
    # 파일명 추출 (확장자 제외)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 이미지 불러오기
    img = cv.imread(file_path)
    file_name="warm_personal"
    cheekimg = cheek(img)
    cv.imwrite("warm_cheek.jpg", cheekimg)
    bar = skin_detector(cheekimg, file_name)
    cv.imwrite("warm_bar.jpg", bar)
    bgr = color_convert2(bar)
    hsv = color_convert(bar)

    color_class = Color()
    skin_type = color_class.color_classifier(hsv)
    cv.imwrite("warm_skin.jpg", bgr)

      # save_img 함수 호출
    save_img(file_name, skin_type)

21.12 74.7
spring
21.3 65.21
autumn
20.51 76.95
spring
15.21 70.43
summer
7.8 20.73
winter
21.07 57.2
autumn
16.32 57.08
winter
18.9 33.47
winter
19.11 41.35
winter
21.61 54.43
autumn
21.43 60.81
autumn
23.57 53.28
autumn
26.82 48.05
autumn
17.45 61.25
winter
28.07 78.68
spring
21.88 48.86
autumn
19.94 69.94
summer
21.45 69.62
spring
15.82 68.23
winter
23.78 56.64
autumn
23.01 75.19
spring
20.68 58.64
autumn
16.78 61.01
winter
19.94 59.48
winter
20.24 65.23
autumn
22.79 29.5
autumn
20.19 52.2
autumn
20.41 66.34
autumn
13.51 63.23
winter
30.58 57.59
autumn
23.01 65.48
autumn
10.36 69.08
summer
19.94 57.51
winter
11.97 66.92
winter
18.05 68.59
winter
30.44 47.71
autumn
17.84 69.28
summer
18.8 69.13
summer
21.45 71.15
spring
19.94 49.2
winter
20.76 54.27
autumn
12.96 67.88
winter
13.6 65.4
winter
11.71 65.39
winter
18.11 69.69
summer
24.79 61.57
autumn
19.62 54.96
winter
18.21 71.67
summer
20.6 64.24
autumn
19.68 59.85
winter


2. 연예인이미지(쿨톤)

In [28]:
import os
import cv2 as cv

def save_img(file_name, skin_type) :

    if skin_type == 0 :
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

# 폴더 경로 지정
folder_path = "/content/drive/MyDrive/사계절_연예인 이미지 데이터셋/prototype/쿨톤_jpg/"

# 폴더 내의 모든 파일을 가져오기 위해 glob 사용
import glob
file_list = glob.glob(folder_path + "*.jpg") + glob.glob(folder_path + "*.JPEG")

for file_path in file_list:
    # 파일명 추출 (확장자 제외)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 이미지 불러오기
    img = cv.imread(file_path)
    file_name="cool_personal"
    cheekimg = cheek(img)
    cv.imwrite("cool_cheek.jpg", cheekimg)
    bar = skin_detector(cheekimg, file_name)
    cv.imwrite("cool_bar.jpg", bar)
    bgr = color_convert2(bar)
    hsv = color_convert(bar)

    color_class = Color()
    skin_type = color_class.color_classifier(hsv)
    cv.imwrite("cool_skin.jpg", bgr)

      # save_img 함수 호출
    save_img(file_name, skin_type)

16.58 65.56
winter
20.34 67.49
autumn
29.92 63.0
autumn
28.54 60.68
autumn
19.01 59.07
winter
27.81 58.88
autumn
13.3 47.75
winter
19.19 72.17
summer
7.76 72.72
summer
21.07 68.68
spring
28.56 57.7
autumn
9.97 69.48
summer
23.74 56.16
autumn
19.94 58.63
winter
21.08 45.61
autumn
10.68 84.24
summer
19.0 64.88
winter
18.18 74.25
summer
29.92 55.09
autumn
18.78 51.63
winter
cool_personal 에러
0.0 12.0
winter
22.31 77.55
spring
8.68 59.43
winter
20.27 25.41
autumn
23.73 76.47
spring
18.16 50.34
winter
24.34 70.43
spring
16.93 55.9
winter
13.38 63.18
winter
10.68 50.34
winter
23.3 59.74
autumn
22.83 37.2
autumn
13.81 65.48
winter
17.95 61.57
winter
17.1 61.02
winter
9.97 75.73
summer
18.41 45.45
winter
21.23 57.17
autumn
21.17 63.93
autumn
19.04 73.5
summer
20.71 79.09
spring
18.8 40.63
winter
20.99 38.5
autumn
22.22 75.4
spring
17.73 60.78
winter
cool_personal 에러
0.0 12.0
winter
19.36 77.79
summer


3. 일반인 이미지(100장)

In [32]:
import os
import cv2 as cv

def save_img(file_name, skin_type) :

    if skin_type == 0 :
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

# 폴더 경로 지정
folder_path = "/content/drive/MyDrive/사계절_연예인 이미지 데이터셋/train data/라벨링용일반인100장/"

# 폴더 내의 모든 파일을 가져오기 위해 glob 사용
import glob
file_list = glob.glob(folder_path + "*.JPG")

for i, file_path in enumerate(file_list):
    # 파일명 추출 (확장자 제외)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 이미지 불러오기
    img = cv.imread(file_path)
    file_name = "people"

    # Process the image and save with unique filenames
    cheekimg = cheek(img)
    cv.imwrite(f"p-cheek_{i}.JPG", cheekimg)
    bar = skin_detector(cheekimg, file_name)
    cv.imwrite(f"p-bar_{i}.JPG", bar)
    bgr = color_convert2(bar)
    hsv = color_convert(bar)

    color_class = Color()
    skin_type = color_class.color_classifier(hsv)
    cv.imwrite(f"p-skin_{i}.JPG", bgr)

    # save_img 함수 호출
    save_img(file_name, skin_type)

27.2 34.79
autumn
25.48 34.82
autumn
19.3 60.53
winter
19.36 51.3
winter
19.0 49.33
winter
16.32 40.36
winter
21.17 21.91
autumn
14.73 43.49
winter
17.1 19.18
winter
20.24 32.24
autumn
27.11 41.63
autumn
25.48 36.71
autumn
21.58 52.14
autumn
24.93 24.7
autumn
22.66 42.92
autumn
11.22 51.54
winter
22.84 38.54
autumn
15.21 57.67
winter
17.1 64.35
winter
24.59 45.69
autumn
19.65 45.96
winter
22.56 35.99
autumn
16.83 48.79
winter
19.94 40.38
winter
21.26 40.37
autumn
21.45 36.12
autumn
23.09 32.17
autumn
18.57 52.07
winter
16.41 65.9
winter
19.36 45.41
winter
24.48 16.05
autumn
18.95 36.61
winter
23.16 30.16
autumn
22.64 58.55
autumn
16.32 48.63
winter
19.04 56.02
winter
20.23 29.7
autumn
18.7 55.09
winter
21.86 25.06
autumn
18.7 50.53
winter
23.62 35.26
autumn
25.41 35.73
autumn
19.38 50.31
winter
21.84 41.0
autumn
15.21 46.92
winter
14.48 54.44
winter
19.27 56.01
winter
22.2 37.88
autumn
23.74 37.89
autumn
22.84 17.76
autumn
19.04 45.33
winter
25.53 17.89
autumn
26.23 31.14
autumn
20.25 